In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
from datasets import Dataset
import datasets

**Here we've used a fine-tuned BERT model with the variation being BERT Base Uncased.**

In [ ]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
#Taking the train.csv dataset as input
url='https://drive.google.com/file/d/1kCoxEbEu17iywZeIh_Hf2jYUPhFY-uKq/view?usp=drive_link'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(dwn_url)
print(df.head())

   Unnamed: 0                                               Text  \
0           0  I can't shake off this constant sense of hopel...   
1           1  I'm constantly second-guessing myself and my d...   
2           2  I'm feeling physically unwell, but I know it's...   
3           3     I'm desperate to escape the overwhelming fear.   
4           4  It's hard to describe the sensation of being t...   

              label  
0        Depression  
1  Anxiety Disorder  
2        Depression  
3    Panic Disorder  
4    Panic Disorder  


In [ ]:
#Taking the test.csv dataset as input for prediction
url='https://drive.google.com/file/d/1FYt6EL4-3HrtLeS6BNsakpdJFQq7uzSw/view?usp=drive_link'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df2= pd.read_csv(dwn_url)
print(df2.head())

   id                                               Text
0   0  I'm worried about the impact of panic on my re...
1   1  Trying to maintain my equilibrium when emotion...
2   2  Feeling like I'm in a constant struggle to fin...
3   3  I'm quick to dismiss others' opinions as less ...
4   4  I hate the person I become when I'm consumed b...


In [ ]:
df.groupby('label').describe()

Unnamed: 0                          \
                                            count        mean         std   
label                                                                       
Anger/ Intermittent Explosive Disorder      154.0  390.811688  224.107898   
Anxiety Disorder                            153.0  412.784314  231.870822   
Depression                                  208.0  374.865385  225.870386   
Narcissistic Disorder                       158.0  401.379747  220.702197   
Panic Disorder                              112.0  383.830357  234.388331   

                                                                            
                                         min     25%    50%     75%    max  
label                                                                       
Anger/ Intermittent Explosive Disorder   9.0  199.75  371.5  588.00  777.0  
Anxiety Disorder                         1.0  212.00  443.0  622.00  779.0  
Depression                               0.0  187.75  351.0  573.50  780.0  
Narcissistic Disorder                   15.0  206.25  417.0  580.75  783.0  
Panic Disorder                           3.0  176.75  399.0  558.75  784.0

In [ ]:
#doing labelEncoder() work manually by replacing every known "label" value with an integer
df['unique_id']=df['label'].apply(lambda x: 1 if x=="Anger/ Intermittent Explosive Disorder" else 2 if x=="Anxiety Disorder" else 3 if x=="Depression" else 4 if x == "Narcissistic Disorder" else 5)

In [ ]:
df.groupby('label').describe()

Unnamed: 0                          \
                                            count        mean         std   
label                                                                       
Anger/ Intermittent Explosive Disorder      154.0  390.811688  224.107898   
Anxiety Disorder                            153.0  412.784314  231.870822   
Depression                                  208.0  374.865385  225.870386   
Narcissistic Disorder                       158.0  401.379747  220.702197   
Panic Disorder                              112.0  383.830357  234.388331   

                                                                            \
                                         min     25%    50%     75%    max   
label                                                                        
Anger/ Intermittent Explosive Disorder   9.0  199.75  371.5  588.00  777.0   
Anxiety Disorder                         1.0  212.00  443.0  622.00  779.0   
Depression                               0.0  187.75  351.0  573.50  780.0   
Narcissistic Disorder                   15.0  206.25  417.0  580.75  783.0   
Panic Disorder                           3.0  176.75  399.0  558.75  784.0   

                                       unique_id                           \
                                           count mean  std  min  25%  50%   
label                                                                       
Anger/ Intermittent Explosive Disorder     154.0  1.0  0.0  1.0  1.0  1.0   
Anxiety Disorder                           153.0  2.0  0.0  2.0  2.0  2.0   
Depression                                 208.0  3.0  0.0  3.0  3.0  3.0   
Narcissistic Disorder                      158.0  4.0  0.0  4.0  4.0  4.0   
Panic Disorder                             112.0  5.0  0.0  5.0  5.0  5.0   

                                                  
                                        75%  max  
label                                             
Anger/ Intermittent Explosive Disorder  1.0  1.0  
Anxiety Disorder                        2.0  2.0  
Depression                              3.0  3.0  
Narcissistic Disorder                   4.0  4.0  
Panic Disorder                          5.0  5.0

In [ ]:
train_df = df[:680].drop("label",axis=1)
test_df = df[680:].drop("label",axis=1)

train_df

,Unnamed: 0,Text,unique_id
0,0,I can't shake off this constant sense of hopel...,3
1,1,I'm constantly second-guessing myself and my d...,2
2,2,"I'm feeling physically unwell, but I know it's...",3
3,3,I'm desperate to escape the overwhelming fear.,5
4,4,It's hard to describe the sensation of being t...,5
...,...,...,...
675,675,I can't seem to quiet the constant 'what if' t...,2
676,676,I feel so empty inside.,3
677,677,It's like I'm trying to navigate through a maz...,1
678,678,Trying to find a way to quiet the inner critic...,2


In [ ]:
train_ds = Dataset.from_dict(train_df)
test_ds = Dataset.from_dict(test_df)

train_ds

Dataset({
    features: ['Unnamed: 0', 'Text', 'unique_id'],
    num_rows: 680
})

In [ ]:
dict_ds = datasets.DatasetDict({"train":train_ds,"test":test_ds})

In [ ]:
dict_ds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Text', 'unique_id'],
        num_rows: 680
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Text', 'unique_id'],
        num_rows: 105
    })
})

In [ ]:
def tokenize(batch):
    return tokenizer(batch["Text"], padding=True, truncation=True)

In [ ]:
dict_ds_encoded = dict_ds.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
dict_ds_encoded

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Text', 'unique_id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 680
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Text', 'unique_id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 105
    })
})

In [ ]:
dict_ds_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'unique_id'])

dict_ds_encoded

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Text', 'unique_id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 680
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Text', 'unique_id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 105
    })
})

In [ ]:
BATCH_SIZE = 16
#This function adds the input_ids, attention_mask and token_type_ids to the encoded datasetdict that is sent to the bert classifier
def order(inp):
    data = list(inp.values())
    print(data)
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

In [ ]:
# converting train split and test split of `dict_ds_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(dict_ds_encoded['train'][:])
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices(dict_ds_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

[<tf.Tensor 'args_3:0' shape=(None,) dtype=int64>, <tf.Tensor 'args_1:0' shape=(None, 27) dtype=int64>, <tf.Tensor 'args_2:0' shape=(None, 27) dtype=int64>, <tf.Tensor 'args_0:0' shape=(None, 27) dtype=int64>]
[<tf.Tensor 'args_3:0' shape=(None,) dtype=int64>, <tf.Tensor 'args_1:0' shape=(None, 25) dtype=int64>, <tf.Tensor 'args_2:0' shape=(None, 25) dtype=int64>, <tf.Tensor 'args_0:0' shape=(None, 25) dtype=int64>]


**Here's the fine-tuned BERT Model with softmax activation**

In [ ]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [ ]:
classifier = BERTForClassification(model, num_classes=6)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = classifier.fit(
    train_dataset,
    epochs=20
)

Epoch 1/20
43/43 [==============================] - 52s 148ms/step - loss: 1.5982 - accuracy: 0.3838
Epoch 2/20
43/43 [==============================] - 4s 84ms/step - loss: 1.0426 - accuracy: 0.6618
Epoch 3/20
43/43 [==============================] - 4s 85ms/step - loss: 0.6894 - accuracy: 0.7721
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.4921 - accuracy: 0.8426
Epoch 5/20
43/43 [==============================] - 4s 84ms/step - loss: 0.3688 - accuracy: 0.8838
Epoch 6/20
43/43 [==============================] - 4s 85ms/step - loss: 0.2755 - accuracy: 0.9132
Epoch 7/20
43/43 [==============================] - 4s 85ms/step - loss: 0.2270 - accuracy: 0.9191
Epoch 8/20
43/43 [==============================] - 4s 93ms/step - loss: 0.1834 - accuracy: 0.9353
Epoch 9/20
43/43 [==============================] - 4s 86ms/step - loss: 0.1402 - accuracy: 0.9471
Epoch 10/20
43/43 [==============================] - 4s 84ms/step - loss: 0.1367 - accuracy: 0.9426
Epoch 1

**Predicting the label value on the test.csv dataset which was previously imported in "df2"**

In [ ]:
text=df2[:]
text_ds = Dataset.from_dict(text)



dict_ds_testing = datasets.DatasetDict({"testing":text_ds})



dict_ds_encoded_testing = dict_ds_testing.map(tokenize, batched=True, batch_size=None)

dict_ds_encoded_testing.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids'])
dict_ds_encoded_testing



  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    testing: Dataset({
        features: ['id', 'Text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 243
    })
})

In [ ]:
def ordertesting(inp):
    data = list(inp.values())

    return {
        'input_ids': data[0],
        'attention_mask': data[1],
        'token_type_ids': data[2]
    }

In [ ]:
BATCH_SIZE = 16
test_dataset_testing = tf.data.Dataset.from_tensor_slices(dict_ds_encoded_testing['testing'][:])
test_dataset_testing = test_dataset_testing.batch(BATCH_SIZE)
test_dataset_testing = test_dataset_testing.map(ordertesting, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
#Predicting values in a numpy array
prediction_new = classifier.predict(test_dataset_testing)
prediction_new

16/16 [==============================] - 3s 36ms/step


array([[8.4451766e-04, 9.0737629e-04, 9.0883002e-03, 2.6541532e-04,
        3.9301387e-05, 9.8885506e-01],
       [1.4618140e-04, 9.7485220e-01, 5.3463079e-04, 2.3784056e-02,
        4.1219281e-04, 2.7075663e-04],
       [1.8706643e-03, 4.9660392e-02, 1.6125973e-01, 7.7607173e-01,
        9.3827387e-03, 1.7548255e-03],
       ...,
       [2.4516173e-04, 1.5665771e-04, 9.9671483e-01, 2.0177805e-04,
        1.0047788e-03, 1.6768383e-03],
       [7.2759838e-04, 3.1196917e-04, 9.7782576e-01, 9.7309344e-04,
        3.3368140e-03, 1.6824765e-02],
       [3.9002788e-04, 7.8406399e-01, 4.7192522e-04, 2.1401685e-01,
        8.3901099e-04, 2.1817574e-04]], dtype=float32)

In [ ]:
#Converting numpy array to pandas dataframe
dataframe=pd.DataFrame(prediction_new)
dataframe.head(6)

,0,1,2,3,4,5
0,0.000845,0.000907,0.009088,0.000265,0.000039,0.988855
1,0.000146,0.974852,0.000535,0.023784,0.000412,0.000271
2,0.001871,0.049660,0.161260,0.776072,0.009383,0.001755
3,0.000315,0.000456,0.002324,0.000559,0.996154,0.000190
4,0.000173,0.998289,0.000085,0.001199,0.000091,0.000164
5,0.000297,0.000451,0.000802,0.000782,0.997490,0.000178


In [ ]:
dataframe.columns = ['0', '1', '2','3','4','5']

In [ ]:
#Dropping "0" column because of really small values that wont affect our data
dataframe.drop('0',axis=1)

,1,2,3,4,5
0,0.000907,0.009088,0.000265,0.000039,0.988855
1,0.974852,0.000535,0.023784,0.000412,0.000271
2,0.049660,0.161260,0.776072,0.009383,0.001755
3,0.000456,0.002324,0.000559,0.996154,0.000190
4,0.998289,0.000085,0.001199,0.000091,0.000164
...,...,...,...,...,...
238,0.000259,0.997981,0.000187,0.000878,0.000500
239,0.000487,0.002373,0.000551,0.996033,0.000188
240,0.000157,0.996715,0.000202,0.001005,0.001677
241,0.000312,0.977826,0.000973,0.003337,0.016825


In [ ]:
def get_max_column(row):
    return row.idxmax()


In [ ]:
#Adding a max_column column to the dataframe that stores which column has the highest value for each corresponding row
dataframe['max_column'] = dataframe.apply(get_max_column, axis=1)
dataframe

,0,1,2,3,4,5,max_column
0,0.000845,0.000907,0.009088,0.000265,0.000039,0.988855,5
1,0.000146,0.974852,0.000535,0.023784,0.000412,0.000271,1
2,0.001871,0.049660,0.161260,0.776072,0.009383,0.001755,3
3,0.000315,0.000456,0.002324,0.000559,0.996154,0.000190,4
4,0.000173,0.998289,0.000085,0.001199,0.000091,0.000164,1
...,...,...,...,...,...,...,...
238,0.000194,0.000259,0.997981,0.000187,0.000878,0.000500,2
239,0.000368,0.000487,0.002373,0.000551,0.996033,0.000188,4
240,0.000245,0.000157,0.996715,0.000202,0.001005,0.001677,2
241,0.000728,0.000312,0.977826,0.000973,0.003337,0.016825,2


In [ ]:
#Dropping every column other than max_column to get the most probable prediction made by the model
dataframe = dataframe.drop(dataframe.columns[dataframe.columns != 'max_column'], axis=1)
dataframe

,max_column
0,5
1,1
2,3
3,4
4,1
...,...
238,2
239,4
240,2
241,2


In [ ]:
text_col=df2["Text"]
text_col

0      I'm worried about the impact of panic on my re...
1      Trying to maintain my equilibrium when emotion...
2      Feeling like I'm in a constant struggle to fin...
3      I'm quick to dismiss others' opinions as less ...
4      I hate the person I become when I'm consumed b...
                             ...                        
238    The pressure to avoid mistakes can keep me fro...
239    Even when challenges arise, I remain unwaverin...
240    It's like my mind is a radio tuned to the freq...
241    It's hard to explain the surge of panic that c...
242    It's hard to explain the inner conflict of man...
Name: Text, Length: 243, dtype: object

In [ ]:
dataframe.insert(0, 'id', range(len(dataframe)))

In [ ]:
dataframe

,id,max_column
0,0,5
1,1,1
2,2,3
3,3,4
4,4,1
...,...,...
238,238,2
239,239,4
240,240,2
241,241,2


In [ ]:
#again doing labelEncoder() manually by replacing integer keys with their actual labels
legend = {
    '1': 'Anger/ Intermittent Explosive Disorder',
    '2': 'Anxiety Disorder',
    '3': 'Depression',
    '4': 'Narcissistic Disorder',
    '5': 'Panic Disorder',
}

dataframe['max_column'].replace(legend, inplace=True)

In [ ]:
dataframe

,id,max_column
0,0,Panic Disorder
1,1,Anger/ Intermittent Explosive Disorder
2,2,Depression
3,3,Narcissistic Disorder
4,4,Anger/ Intermittent Explosive Disorder
...,...,...
238,238,Anxiety Disorder
239,239,Narcissistic Disorder
240,240,Anxiety Disorder
241,241,Anxiety Disorder


In [ ]:
#Adding the Text column for manual verification purposes
dataframe.insert(1, 'Text', text_col)
dataframe

,id,Text,max_column
0,0,I'm worried about the impact of panic on my re...,Panic Disorder
1,1,Trying to maintain my equilibrium when emotion...,Anger/ Intermittent Explosive Disorder
2,2,Feeling like I'm in a constant struggle to fin...,Depression
3,3,I'm quick to dismiss others' opinions as less ...,Narcissistic Disorder
4,4,I hate the person I become when I'm consumed b...,Anger/ Intermittent Explosive Disorder
...,...,...,...
238,238,The pressure to avoid mistakes can keep me fro...,Anxiety Disorder
239,239,"Even when challenges arise, I remain unwaverin...",Narcissistic Disorder
240,240,It's like my mind is a radio tuned to the freq...,Anxiety Disorder
241,241,It's hard to explain the surge of panic that c...,Anxiety Disorder


In [ ]:
dataframe.rename(columns={'max_column': 'label'}, inplace=True)

In [ ]:
dataframe

,id,Text,label
0,0,I'm worried about the impact of panic on my re...,Panic Disorder
1,1,Trying to maintain my equilibrium when emotion...,Anger/ Intermittent Explosive Disorder
2,2,Feeling like I'm in a constant struggle to fin...,Depression
3,3,I'm quick to dismiss others' opinions as less ...,Narcissistic Disorder
4,4,I hate the person I become when I'm consumed b...,Anger/ Intermittent Explosive Disorder
...,...,...,...
238,238,The pressure to avoid mistakes can keep me fro...,Anxiety Disorder
239,239,"Even when challenges arise, I remain unwaverin...",Narcissistic Disorder
240,240,It's like my mind is a radio tuned to the freq...,Anxiety Disorder
241,241,It's hard to explain the surge of panic that c...,Anxiety Disorder


In [ ]:
dataframe.groupby('label').describe()

id                               \
                                       count        mean        std   min   
label                                                                       
Anger/ Intermittent Explosive Disorder  46.0  118.065217  73.654570   1.0   
Anxiety Disorder                        61.0  135.803279  60.248878  11.0   
Depression                              49.0  119.224490  69.643217   2.0   
Narcissistic Disorder                   47.0  112.361702  73.798912   3.0   
Panic Disorder                          40.0  114.125000  76.986242   0.0   

                                                                     
                                          25%    50%     75%    max  
label                                                                
Anger/ Intermittent Explosive Disorder  43.75  124.5  178.25  242.0  
Anxiety Disorder                        92.00  137.0  177.00  241.0  
Depression                              61.00  117.0  181.00  227.0  
Narcissistic Disorder                   53.50  102.0  186.00  239.0  
Panic Disorder                          45.75   96.5  188.50  237.0

In [ ]:
#Deleting the Text column for submission purposes
dataframe.drop("Text",axis=1)

,id,label
0,0,Panic Disorder
1,1,Anger/ Intermittent Explosive Disorder
2,2,Depression
3,3,Narcissistic Disorder
4,4,Anger/ Intermittent Explosive Disorder
...,...,...
238,238,Anxiety Disorder
239,239,Narcissistic Disorder
240,240,Anxiety Disorder
241,241,Anxiety Disorder


In [ ]:
dataframe

,id,Text,label
0,0,I'm worried about the impact of panic on my re...,Panic Disorder
1,1,Trying to maintain my equilibrium when emotion...,Anger/ Intermittent Explosive Disorder
2,2,Feeling like I'm in a constant struggle to fin...,Depression
3,3,I'm quick to dismiss others' opinions as less ...,Narcissistic Disorder
4,4,I hate the person I become when I'm consumed b...,Anger/ Intermittent Explosive Disorder
...,...,...,...
238,238,The pressure to avoid mistakes can keep me fro...,Anxiety Disorder
239,239,"Even when challenges arise, I remain unwaverin...",Narcissistic Disorder
240,240,It's like my mind is a radio tuned to the freq...,Anxiety Disorder
241,241,It's hard to explain the surge of panic that c...,Anxiety Disorder


In [ ]:
dataframe.to_csv('problemstatement1_test6.csv', sep=',', encoding='utf-8',index=False)